Q=AW_Q 
K=AW_K
V=AW_V

In [8]:
import math
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l

In [9]:
def sequence_mask(X, valid_len, value = 0):
    maxlen = X.size(1)
    mask = torch.arange((maxlen), dtype = torch.float32, device = X.device)[None, :] < valid_len[:, None]
    X[~mask] = value # ~mask取反，将无效特征位置设为value（默认0）
    return X

生成0到maxlen-1的序列（特征位置索引），形状为(1, maxlen)
arange = torch.arange(maxlen, dtype=torch.float32, device=X.device)[None, :]

valid_len[:, None] 将(6,)的valid_len扩展为(6, 1)，适配特征维度
mask = arange < valid_len[:, None]  # 形状为(6, maxlen)，即(6, 4)

In [10]:
def masked_softmax(X, valid_lens):
    if valid_lens is None:
        return nn.functional.softmax(X, dim = -1)
    else:
        shape = X.shape
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
        else:
            valid_lens = valid_lens.reshape(-1)
        X = sequence_mask(X.reshape(-1, shape[-1]), valid_lens, value = -1e6)
        return nn.functional.softmax(X.reshape(shape), dim = -1)

In [11]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([2, 3]))

tensor([[[0.5050, 0.4950, 0.0000, 0.0000],
         [0.5753, 0.4247, 0.0000, 0.0000]],

        [[0.3858, 0.3640, 0.2502, 0.0000],
         [0.4396, 0.3210, 0.2394, 0.0000]]])

In [12]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([[1, 3], [2, 4]]))

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.4643, 0.3452, 0.1905, 0.0000]],

        [[0.4110, 0.5890, 0.0000, 0.0000],
         [0.3208, 0.3391, 0.1516, 0.1885]]])

若 valid_lens 是 1D（每个样本一个有效长度）：通过 repeat_interleave 重复 shape[1] 次（shape[1] 是序列长度），让每个位置都对应相同的有效长度。
valid_lens 是 1D 张量，形状为 (batch_size,)，例如 valid_lens = torch.tensor([2, 3])（表示 batch 中两个样本的有效长度分别为 2 和 3）。
shape[1] 是输入 X 的序列长度（第二维），例如 shape = (2, 4, 5)，则 shape[1] = 4（每个样本包含 4 个序列元素）。
原 valid_lens 中的第一个元素 2 被重复 shape[1] = 4 次 → [2, 2, 2, 2]。
原 valid_lens 中的第二个元素 3 被重复 shape[1] = 4 次 → [3, 3, 3, 3]。
tensor([2, 2, 2, 2, 3, 3, 3, 3])

In [17]:
class DotProductAttention(nn.Module):
    def __init__(self, dropout, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)

    def forward(self, queries, key, values, valid_lens = None):
        d = queries.shape[-1]
        scores = torch.bmm(queries, keys.transpose(1, 2)) / math.sqrt(d)
        self.attention_weights = masked_softmax(scores, valid_lens)
        return torch.bmm(self.dropout(self.attention_weights), values)

简单说：bmm 是 “专用批量矩阵乘法工具”，而 matmul 是 “通用矩阵乘法瑞士军刀”。在明确处理 3 维批量矩阵且无需广播时，bmm 更直观；其他情况（尤其是高维或需要广播时）用 matmul。

在 Python 中，**kwargs 是一种特殊的参数语法，用于处理函数调用时传入的关键字参数（key-value 形式的参数），并将这些参数打包成一个字典（dictionary）。

在 PyTorch 中，torch.matmul(queries, keys.transpose(-2, -1)) 的矩阵乘法维度匹配遵循最后两个维度相乘的规则，这与多头注意力中 “每个头独立计算注意力分数” 的逻辑完全契合。

In [18]:
queries = torch.normal(0, 1, (2, 1, 2))
keys = torch.ones((2, 10, 2))
values = torch.arange(40, dtype=torch.float32).reshape(1, 10, 4).repeat(
    2, 1, 1)
valid_lens = torch.tensor([2, 6])
attention = DotProductAttention(dropout = 0.5)
attention.eval()
attention(queries, keys, values, valid_lens)

tensor([[[ 2.0000,  3.0000,  4.0000,  5.0000]],

        [[10.0000, 11.0000, 12.0000, 13.0000]]])

最后 .repeat(2, 1, 1)：
对张量进行重复操作，参数 (2, 1, 1) 表示：
第一个维度重复 2 次
第二个维度重复 1 次（保持不变）
第三个维度重复 1 次（保持不变）
最终得到的张量形状为 (2, 10, 4)

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, key_size, query_size, value_size, num_hiddens, num_heads, dropout, bias = False, **kwargs):
        super(MutiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias = bias) # query_size 指的是 输入X的 最后一维
        self.W_k = nn.Linear(key_size, num_hiddens, bias = bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias =bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias = bias)

    def forward(self, queries, keys, values, valid_lens):

In [6]:
# 基于位置的前馈网络
class PositionWiseFFN(nn.Module):
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs, **kwargs):
        super(PositionWiseFFN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)

    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))

In [8]:
ffn = PositionWiseFFN(4, 4, 8)
ffn.eval()
ffn(torch.ones((2, 3, 4))).shape

torch.Size([2, 3, 8])

在 PyTorch 中，nn.Linear 层（全连接层）虽然本质上处理的是二维特征（[batch_size, feature_dim]），但它对输入的前导维度（batch 维度）具有兼容性，会自动忽略除最后一个维度之外的所有维度，只对最后一个维度进行线性变换。

只关注输入的最后一个维度（即 feature_dim=4），前面的所有维度（2 和 3）会被视为 “批量维度”。

In [9]:
ffn(torch.ones((2, 3, 4)))[0]

tensor([[ 0.3013,  0.2407,  0.1331,  0.2597, -0.1420,  0.2759, -0.2957,  0.0645],
        [ 0.3013,  0.2407,  0.1331,  0.2597, -0.1420,  0.2759, -0.2957,  0.0645],
        [ 0.3013,  0.2407,  0.1331,  0.2597, -0.1420,  0.2759, -0.2957,  0.0645]],
       grad_fn=<SelectBackward0>)

In [10]:
# 对比不同维度的层规范化和批量规范化的效果
ln = nn.LayerNorm(2)
bn = nn.BatchNorm1d(2)
X = torch.tensor([[1, 2], [2, 3]], dtype = torch.float32)
print('layer norm:', ln(X), '\nbatch norm:', bn(X))

layer norm: tensor([[-1.0000,  1.0000],
        [-1.0000,  1.0000]], grad_fn=<NativeLayerNormBackward0>) 
batch norm: tensor([[-1.0000, -1.0000],
        [ 1.0000,  1.0000]], grad_fn=<NativeBatchNormBackward0>)


type：描述变量本身的类型（即变量是哪种对象）。
例如：整数、字符串、列表，或 PyTorch 中的 Tensor、nn.Module 等。


dtype：仅用于数值型对象（如数组、张量），描述对象内部存储的数据的类型（即元素的数值类型）。
例如：整数是 32 位还是 64 位，浮点数是单精度还是双精度等。

In [14]:
# 残差连接和层规范化
class AddNorm(nn.Module):
    def __init__(self, normalized_shape, dropout, **kwargs):
        super(AddNorm, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)

    def forward(self, X, y):
        return self.ln(self.dropout(y) + X)        

In [15]:
add_norm = AddNorm([3, 4], 0.5)
add_norm.eval()
add_norm(torch.ones((2, 3, 4)), torch.ones((2, 3, 4))).shape

torch.Size([2, 3, 4])

必须确保 normalized_shape 与输入张量的最后 N 个维度完全匹配（N 是 normalized_shape 的长度），否则会报错。

例如：

若输入是 (32, 50, 256)，normalized_shape=256（或 (256,)）是正确的；
若错误设为 normalized_shape=50，则会因 “输入最后 1 个维度是 256，与 50 不匹配” 报错。

(256,) 表示一个只包含一个元素的元组（tuple），其中唯一的元素是整数 256。

这个写法的关键是末尾的逗号 ,，它用来区分 “单个元素的元组” 和 “用括号包裹的普通数值”：

(256) 只是带括号的整数 256，不是元组
(256,) 才是包含 256 这一个元素的元组

In [17]:
# 编码器
class EncoderBlock(nn.Module):
    def __init__(self, key_size, query_size, value_size, num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens, num_heads, dropout, use_bias=False, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        self.attention = multiHeadAttention(key_size, query_size, value_size, num_hiddens, num_heads, dropout, use_bias)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFFN(ffn_num_input, ffn_num_hiddens, ffn_num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)

    def forward(self, X, valid_lens):
        Y = self.addnorm1(X, self.attention(X, X, X, valid_lens))
        return self.addnorm2(Y, self.ffn(Y))

In [18]:
X = torch.ones((2, 100, 24))
valid_lens = torch.tensor([3, 2])
encoder_blk = EncoderBlock(24, 24, 24, 24, [100, 24], 24, 48, 8, 0.5)
encoder_blk.eval()
encoder_blk(X, valid_lens).shape

NameError: name 'multiHeadAttention' is not defined